# 2D Elliptic equations

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt

## Demo of array <-> vector

In [ ]:
A = np.array( [[1, 2, 3], [4, 5, 6]])
print(A)
print()
B = A.flatten()
print(B)
print()
print(B.reshape(2,3))

## An elliptic solver

In [ ]:
# solution coefficients
sol_a = 2.8
sol_b = -1.1
sol_c = 1.8
sol_d = 2.2

def sol(x, y):
    "True solution"
    return np.sin(sol_a*x + sol_b)*np.cos(sol_c*y + sol_d)

def f(x, y):
    "Right-hand side function"
    return -(sol_a**2 + sol_c**2)*sol(x, y)

# grid coefficients
x_l, x_u = -1.8, 2.3
y_l, y_u = 0.3, 4.8
n_x, n_y = 101, 103

# grid spacing
dx = (x_u-x_l)/(n_x-1)
dy = (y_u-y_l)/(n_y-1)

# 1D grids
x = np.linspace(x_l, x_u, n_x)
y = np.linspace(y_l, y_u, n_y)

# 2D grids
X, Y = np.meshgrid(x, y)

print(X)
print()
print(Y)

In [ ]:
plt.figure(figsize=(5,5))
plt.contour(x, y, sol(X, Y));

In [ ]:
# number of unknowns
n_unk = n_x*n_y

# initialize right-hand side
b = np.zeros(n_unk)

# set up the Poisson system
I, J, data = [], [], []

# "lazy way" do do the loop
for i in range(n_y):
    yi = y[i]
    for j in range(n_x):
        xj = x[j]
        
        k = i*n_x + j
        
        # check if this is a boundary point
        if i == 0 or i == n_y-1 or j == 0 or j == n_x-1:
            I.append(k)
            J.append(k)
            data.append(1)
            b[k] = sol(xj, yi)
        else:
            I.extend([k, k, k, k, k])
            J.extend([k-n_x, k-1, k, k+1, k+n_x])
            data.extend([1/dy**2, 1/dx**2, -2/dx**2 -2/dy**2, 1/dx**2, 1/dy**2])
            b[k] = f(xj, yi)
            
# build the sparse matrix
A = sp.csr_matrix((data, (I,J)), (n_unk, n_unk))

plt.spy(A[:3*n_x,:3*n_x], markersize=1);
#plt.spy(A, markersize=1);

In [ ]:
u = sla.spsolve(A, b)

plt.figure(figsize=(5,5))
plt.contour(x, y, u.reshape(n_y, n_x));

In [ ]:
plt.figure(figsize=(6,5))
plt.contour(x, y, u.reshape(n_y, n_x)-sol(X,Y))
plt.colorbar()
print(np.max(np.abs(u.reshape(n_y, n_x)-sol(X,Y))))